In [1]:
#Libraries Used
import socket
import pickle
import time
import sys
import pdfplumber
import hashlib
from fpdf import FPDF

In [2]:
with open('publicKey_registrar.pkl','rb') as file:
    publicKey_registrar = pickle.load(file)

with open('publicKey_director.pkl','rb') as file:
    publicKey_director = pickle.load(file)
    
with open('publicKey_student.pkl','rb') as file:
    publicKey_student = pickle.load(file)

with open('privateKey_student.pkl','rb') as file:
    privateKey_student = pickle.load(file)

In [3]:
#Evaluating the binary exponention 
#M^e (mod n)
#C^d (mod n)

def binary_modular_exponention(m,e,n):
    result = 1
    m = m % n
    if (m == 0):
        return 0
    while(e>0):
        if(e&1):
            result = (result*m) % n
        e = e>>1
        m = (m*m) % n
    return result

In [4]:
#Encryption Algorithm
def encrypt_data(encryption_key, message):
    cipher_text = ''
    for value in range(0, len(message)):
        encrypted_char = binary_modular_exponention(ord(message[value]), encryption_key[0], encryption_key[1])
        cipher_text += chr(encrypted_char)
    return cipher_text

In [5]:
#Decryption Algorithm
def decrypt_data(decryption_key, cipher_data):
    original_text = ''
    for value in range(0, len(cipher_data)):
        decrypted_char = binary_modular_exponention(ord(cipher_data[value]), decryption_key[0], decryption_key[1])
        original_text += chr(decrypted_char)
    return original_text

In [6]:
#Closing any previous socket connection
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.close()

In [7]:
#Defining the local host and different ports
host = 'local host'
server_port = 5000

#Defining the Socket Connection
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
print('Connecting to Server....')
s.connect(('127.0.0.1', server_port))
print()
print('Connected to Server')

Connecting to Server....

Connected to Server


In [8]:
#Connected to Server
msg = s.recv(1024)
print(msg.decode())

#Message Generation Step 1
roll_no = str(input("Enter your Roll Number: "))
request_type = str(input("Enter your request about the type of document. Enter 1 for Grade Card or 2 for Degree Certificate: "))

if request_type == "1":
    roll_no = "Grade_" + roll_no
else:
    roll_no = "Degree_" + roll_no
    
s.send(roll_no.encode())
print('Request sent by Client :',roll_no)
print()
msg = s.recv(1024)
print('Message received from Server(before decryption) : ',msg.decode())
print() 
try:
    #Step 5
    #Client receives the message
    received_data_encrypted = msg.decode()
    #Decrypting the Message
    received_data = decrypt_data(privateKey_student, received_data_encrypted)
    print('Message received from Server(after decryption) : ',received_data)
    print() 
    
    #Splitting the data and Checking the Hash values
    grade_value = received_data.split('||')[0] + "||" + received_data.split('||')[1] + "||" + received_data.split('||')[2]
    received_hash = received_data.split('||')[3]    
    print("Received Hashed Value: ",received_hash)
    print()
    
    #Decrypts the received hash using public key of registrar and director
    double_decrypted_hash = decrypt_data(publicKey_director, received_hash)
    decrypted_hash = decrypt_data(publicKey_registrar, double_decrypted_hash)
    print("Decrypted Hashed Value: ",decrypted_hash)
    print()
    
    #Step 6
    #Finding And Comparing Hashes
    calculated_hash = hashlib.sha256(grade_value.encode())
    print("Calculated Hashed Value: ",calculated_hash.hexdigest())
    print()

    
    #Comparing Hashes
    if (decrypted_hash == calculated_hash.hexdigest()): #Authentic Message
        print("Original Text Received. Valid Signature!")
        print()
        grade = grade_value.split('||')[0]
        print("Your Grade Report")
        print(grade)
        print()
        #Creating a pdf and storing it
        print("Storing the report as a .pdf file")
        grade_pdf = FPDF()
        # Add a page
        grade_pdf.add_page()
        # Setting style and size of font 
        grade_pdf.set_font("Arial", size = 15)
        grade_pdf.multi_cell(200, 10, txt = grade, align = 'L')
        # Saving the pdf
        name_pdf = "Received_" + roll_no + ".pdf"
        grade_pdf.output(name_pdf) 
    else:
        print("No, original data is not received. Data has been hampered in between. Signature is not valid!")

except Exception as e:
    print("Roll Number does not exist!")
    print(e)
    pass

Connected succcessfully to Server
Enter your Roll Number: MT20054
Enter your request about the type of document. Enter 1 for Grade Card or 2 for Degree Certificate: 1
Request sent by Client : Grade_MT20054

Message received from Server(before decryption) :  N|bIKy|b;-E
\L~I)I, ,,t;~~ww,$t)>IdL8w)>>L~,,; )~LLI,L  dt>Icb3(d3	2GGkE(GG8~\$t 

Message received from Server(after decryption) :  Name: Shreya Goel
Grade: A||time.struct_time(tm_year=2021, tm_mon=11, tm_mday=17, tm_hour=19, tm_min=46, tm_sec=18, tm_wday=2, tm_yday=321, tm_isdst=0)||Delhi||8qKbu78uS%G%cu6qSScbccGqq%%cKSd68%6qccGbq66Sc6bbdS

Received Hashed Value:  8qKbu78uS%G%cu6qSScbccGqq%%cKSd68%6qccGbq66Sc6bbdS

Decrypted Hashed Value:  80513b47849fafc4560929cbcc3a00ffc132e9d68f2ee60ccab20669c6bbd3e9

Calculated Hashed Value:  80513b47849fafc4560929cbcc3a00ffc132e9d68f2ee60ccab20669c6bbd3e9

Original Text Received. Valid Signature!

Your Grade Report
Name: Shreya Goel
Grade: A

Storing t